In [10]:
import pandas as pd
import numpy as np
import sys

In [11]:
repodb_df = pd.read_csv('data/repo_db.csv')
repodb_df['drug_name'] = repodb_df['drug_name'].str.lower()

In [12]:
ourdata = pd.read_csv('data/drug_riskscores.csv')
ourdata['drug'] = ourdata['drug'].str.lower()

Drug Overlap between our data and RepoDB

In [13]:
drug_overlap_repodb = ourdata.join(repodb_df.set_index(['drug_name']), how='inner', on=['drug'], lsuffix='_x', rsuffix='_y')
drug_overlap_repodb = drug_overlap_repodb.iloc[: , 1:]
#drug_overlap_repodb.to_csv('drug_overlap_repodb.csv')
drug_overlap_repodb

,drug,risk_score,drug_id,ind_name,ind_id,NCT,status,phase,DetailedStatus
15,abarelix,0.189733,DB00106,Ischemic priapism,C2711256,NaN,Approved,NaN,NaN
16,anidulafungin,0.132178,DB00362,Candidiasis of the esophagus,C0239295,NaN,Approved,NaN,NaN
16,anidulafungin,0.132178,DB00362,candidal peritonitis,C2062982,NaN,Approved,NaN,NaN
16,anidulafungin,0.132178,DB00362,Candidemia,C0877445,NaN,Approved,NaN,NaN
16,anidulafungin,0.132178,DB00362,Fungemia,C0085082,NCT00806351,Terminated,Phase 3,The study was prematurely terminated on May 18...
...,...,...,...,...,...,...,...,...,...
807,zonisamide,0.039542,DB00909,Epilepsy,C0014544,NCT00692003,Terminated,Phase 3,Sponsor's decision
807,zonisamide,0.039542,DB00909,"Alcoholic Intoxication, Chronic",C0001973,NCT00862563,Terminated,Phase 2,Recruitment goals could not be met before endi...
807,zonisamide,0.039542,DB00909,alcohol use disorder,C0001956,NCT01566370,Terminated,Phase 2,recruitment infeasible
807,zonisamide,0.039542,DB00909,Bipolar Disorder,C0005586,NCT01566370,Terminated,Phase 2,recruitment infeasible


Drug Overlap between our data and BROAD

In [14]:
broad_df = pd.read_csv('data/broad_repurp_data_orig.tsv',sep='\t',header=None)
broad_df = broad_df.drop(range(9))

new_header = broad_df.iloc[0] 
broad_df = broad_df[1:]
broad_df.columns = new_header

broad_df['pert_iname'] = broad_df['pert_iname'].str.lower()

In [15]:
drug_overlap_broad = ourdata.join(broad_df.set_index(['pert_iname']), how='inner', on=['drug'], lsuffix='_x', rsuffix='_y')
drug_overlap_broad = drug_overlap_broad.iloc[: , 1:]
#drug_overlap_broad.to_csv('drug_overlap_broad.csv')
drug_overlap_broad

,drug,risk_score,clinical_phase,moa,target,disease_area,indication
15,abarelix,0.189733,Phase 3,gonadotropin releasing factor hormone receptor...,GNRHR,NaN,NaN
16,anidulafungin,0.132178,Launched,"fungal 1,3-beta-D-glucan synthase inhibitor",NaN,infectious disease,candidemia|fungal infection|esophageal candidi...
19,bivalirudin,0.039756,Launched,thrombin inhibitor,F2,cardiology,angina pectoris
23,carfilzomib,0.076173,Launched,proteasome inhibitor,PSMA1|PSMA2|PSMA3|PSMA4|PSMA5|PSMA6|PSMA7|PSMA...,hematologic malignancy,multiple myeloma
28,eptifibatide,0.013718,Launched,platelet aggregation inhibitor,ITGA2B|ITGB3,cardiology,acute coronary syndrome (ACS)|percutaneous cor...
...,...,...,...,...,...,...,...
800,zidovudine,0.293031,Launched,nucleoside reverse transcriptase inhibitor,TERT,infectious disease,human immunodeficiency virus (HIV-1)|acquired ...
802,zileuton,0.087954,Launched,leukotriene synthesis inhibitor|lipoxygenase i...,ALOX5,pulmonary,asthma
803,ziprasidone,0.020284,Launched,dopamine receptor antagonist|serotonin recepto...,ADRA1A|ADRA1B|ADRA2A|ADRA2B|ADRA2C|CHRM1|CHRM2...,neurology/psychiatry,schizophrenia|bipolar disorder
805,zolmitriptan,0.021104,Launched,serotonin receptor agonist,HTR1A|HTR1B|HTR1D|HTR1E|HTR1F,neurology/psychiatry,migraine headache


Indications overlap between DDR and RepoDB

In [18]:
ddr = pd.read_csv('validation data/DDR_full_net.csv')
ddr

,drug_a,clinical_phase_a,moa_a,target_a,disease_area_a,indication_a,risk_score_a,scaled_risk_score,total_overlaps,drug_b,clinical_phase_b,moa_b,target_b,disease_area_b,indication_b,risk_score_b,gene
0,abiraterone,Launched,androgen biosynthesis inhibitor,CYP11B1|CYP17A1,oncology,prostate cancer,0.047161,2.578095e-04,1,mitotane,Launched,antineoplastic agent,CYP11A1|CYP11B1|ESR1,oncology,adrenal cortical carcinoma,0.197996,CYP11B1
1,abiraterone,Launched,androgen biosynthesis inhibitor,CYP11B1|CYP17A1,oncology,prostate cancer,0.047161,8.591064e-04,1,progesterone,Launched,progesterone receptor agonist,CATSPER1|CATSPER2|CATSPER3|CATSPER4|CYP17A1|ES...,obstetrics/gynecology|endocrinology,infertility|amenorrhea,0.078474,CYP17A1
2,acamprosate,Launched,glutamate receptor antagonist,GABRA1|GABRA2|GABRA3|GABRA4|GABRA5|GABRA6|GABR...,neurology/psychiatry,abstinence from alcohol,0.048947,1.340000e-39,16,amoxapine,Launched,norepinephrine reputake inhibitor,ADRA1A|ADRA1B|ADRA1D|ADRA2A|ADRA2B|ADRA2C|CHRM...,neurology/psychiatry,depression,0.113536,GABRA1
3,acamprosate,Launched,glutamate receptor antagonist,GABRA1|GABRA2|GABRA3|GABRA4|GABRA5|GABRA6|GABR...,neurology/psychiatry,abstinence from alcohol,0.048947,1.340000e-39,16,amoxapine,Launched,norepinephrine reputake inhibitor,ADRA1A|ADRA1B|ADRA1D|ADRA2A|ADRA2B|ADRA2C|CHRM...,neurology/psychiatry,depression,0.113536,GABRA2
4,acamprosate,Launched,glutamate receptor antagonist,GABRA1|GABRA2|GABRA3|GABRA4|GABRA5|GABRA6|GABR...,neurology/psychiatry,abstinence from alcohol,0.048947,1.340000e-39,16,amoxapine,Launched,norepinephrine reputake inhibitor,ADRA1A|ADRA1B|ADRA1D|ADRA2A|ADRA2B|ADRA2C|CHRM...,neurology/psychiatry,depression,0.113536,GABRA3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18737,zonisamide,Launched,sodium channel blocker|T-type calcium channel ...,CA1|CA10|CA11|CA12|CA13|CA14|CA2|CA3|CA4|CA5A|...,neurology/psychiatry,seizures|epilepsy,0.039542,3.020000e-34,15,topiramate,Launched,carbonic anhydrase inhibitor|glutamate recepto...,CA1|CA12|CA2|CA4|CA7|GABRA1|GABRA2|GABRA3|GABR...,neurology/psychiatry,epilepsy|migraine headache,0.041200,SCN9A
18738,zonisamide,Launched,sodium channel blocker|T-type calcium channel ...,CA1|CA10|CA11|CA12|CA13|CA14|CA2|CA3|CA4|CA5A|...,neurology/psychiatry,seizures|epilepsy,0.039542,2.917948e-03,1,valdecoxib,Withdrawn,cyclooxygenase inhibitor,CA12|PTGS2,NaN,NaN,0.033806,CA12
18739,zonisamide,Launched,sodium channel blocker|T-type calcium channel ...,CA1|CA10|CA11|CA12|CA13|CA14|CA2|CA3|CA4|CA5A|...,neurology/psychiatry,seizures|epilepsy,0.039542,3.700000e-06,3,verapamil,Launched,calcium channel blocker,CACNA1A|CACNA1B|CACNA1C|CACNA1D|CACNA1F|CACNA1...,cardiology,hypertension,0.025678,CACNA1G
18740,zonisamide,Launched,sodium channel blocker|T-type calcium channel ...,CA1|CA10|CA11|CA12|CA13|CA14|CA2|CA3|CA4|CA5A|...,neurology/psychiatry,seizures|epilepsy,0.039542,3.700000e-06,3,verapamil,Launched,calcium channel blocker,CACNA1A|CACNA1B|CACNA1C|CACNA1D|CACNA1F|CACNA1...,cardiology,hypertension,0.025678,CACNA1I


In [40]:
a = drug_overlap_repodb['ind_name']
repodb_ind = list(map(lambda x: x.lower(), a))
repodb_ind = np.unique(repodb_ind)

In [41]:
b = list(ddr['indication_a'])
b = list(map(lambda x: str(x).lower().split('|'), b))
b = [item for sublist in b for item in sublist]
ddr_ind = np.unique(b)

In [49]:
intersection = set(ddr_ind) & set(repodb_ind)
ddr_ind_remainder = set(ddr_ind) - set(repodb_ind)
repodb_ind_remainder = set(repodb_ind) - set(ddr_ind)

print(len(intersection))
print(len(ddr_ind_remainder))
print(len(repodb_ind_remainder))

print('{}% of overlapping indication names'.format((84 / (186 + 1383 + 84) ) * 100 ))

84
186
1383
5.081669691470054% of overlapping indication names
